In [1]:
import matplotlib
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score
from IPython.display import Image, HTML
import matplotlib.pyplot as plt
import pylab
import glob, os
from IPython.display import clear_output
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import grid_search
import datetime
from datetime import datetime
import xgboost as xgb
from sklearn.metrics import log_loss


way = "G:\!datascience\Walmart"
train_save_name1 = 'train_save_1.csv'
train_save_name2 = 'train_save_2.csv'
test_save_name2 = 'test_save_2.csv'

tt4_save_name1 = 'tt4_save_1.csv'
tat_save_name1 = 'tat_save_1.csv'

#настройки показа в панде
pd.options.display.max_columns = 100 #def100
pd.options.display.max_colwidth = 120 #def100
pd.options.display.max_rows = 100 #def 100


##1 Data preparation

In [2]:
# model 1
# traintable = VisitNumber x attributes

# new feature 1 - total items purhased, clustarization and visualoization of distribution and corr with types 2 - if any were -1
# new feature 3 - number of departments with purchased goods
# split 60-20-20 train CV test
# run feature importance - get know how it works. CV leave one out feature and check accuracy change
# correlation of onehotencoded olums with triptypecolumn
# forum
# most frequent errors from confusion matrix
# check for the frequencies of each Department / number / UPC and its correlations with some classes
#and another Depatments / UPcs etc

# MENS WEAR and MENSWEAR - unite columns?

# бухло и конфеты?

# 1) delete UPC
# 2) sum ScanCount over DepartmentDescription
# 2) onehot Department Description 
# 3) onehotweekday

# NaN management

# a lot of potential in feature selection from UPC

#PREDICTION probability format

# replace Nans . UPC and other
# step1 - without UPC, step2 - with UPC
# step3 - onehot FinelineNumber
# сделать метрику ошибки+
# более крупные категории?
# возврат товара - отдельный тип визита
# 14 класс встречается всео 3 раза и xgboost его даже не предсказывает на CV

#adding weekday - decreas in quality?
# tt5 RF acc = 0.614 CV
# xgb tt acc 0.616 CV
# RF tt acc 0.64 CV
# xgb tt5 acc 0.643 CV, runtime 15hours on 1 core


# code hints train_raw[train_raw['Upc'].isin([834,3032])]

In [2]:
test_raw = pd.read_csv(filepath_or_buffer = way + str('\\test.csv'), sep =",")
train_raw = pd.read_csv(filepath_or_buffer = way + str('\\train.csv'), sep =",")
sample_submission = pd.read_csv(filepath_or_buffer = way + str('\\sample_submission.csv'), sep =",")

train = train_raw
train = train.fillna('other')
test = test_raw
test = test.fillna('other')

# one-hot
saved_types = {}
for i in train.DepartmentDescription.unique():
    saved_types['DepartmentDescription=' + i] = np.int8
for i in train.Weekday.unique():
    saved_types['Weekday=' + i] = np.int8
saved_types['TType']=np.object
       
col_wkd = ['Weekday=' + i for i in train_raw['Weekday'].unique()]  
    
train['TType']=1 #Table Type
test['TType']=0 
test['TripType'] = -1
tat1 = pd.concat([train, test])#test and train

tat = pd.read_csv(filepath_or_buffer=tat_save_name1,dtype = saved_types)

In [3]:
col_fln = tat['FinelineNumber'].unique()
dtypes_tt4 = {}
for i in col_fln:
    dtypes_tt4[i]=np.int8
dtypes_tt4['VisitNumber']=np.int64

In [27]:
# OHE runned 1 time
# tat = pd.concat([train, test])#test and train
# tat = tat.reset_index(drop = True)
# tat1 = pd.concat([train, test])#test and train

# from sklearn.feature_extraction import DictVectorizer
# vec = DictVectorizer()

# a = tat[['Weekday','DepartmentDescription']].to_dict(orient = 'records')
# b = vec.fit_transform(a).toarray()
# c = vec.get_feature_names()
# d = pd.DataFrame(data = b, columns = c)
# tat = pd.concat([tat.drop(['Weekday','DepartmentDescription'],1),d],axis=1,join='inner')
# new_col = c

# # tat.to_csv(path_or_buf = tat_save_name1, index = False )

In [4]:
# createing training table (tt)

ze = np.zeros((len(tat1.VisitNumber.unique()),), dtype=np.int8)
tt = pd.DataFrame(data = {'VisitNumber':tat1.VisitNumber.unique().astype(np.int64)})
# for i in saved_types:
#     tt[i]=ze
tt.index=tt.VisitNumber
tt2  = pd.pivot_table(tat1, values='ScanCount', index=['VisitNumber'],
                   columns=['DepartmentDescription'], aggfunc=np.sum)
tt2 = tt2.fillna(0).astype(np.int8)
tt2.columns = ['DepartmentDescription=' + i for i in list(tt2.columns)]
tt_ind = pd.pivot_table(tat1, values='TripType', index=['VisitNumber'], aggfunc=np.mean)
tt = pd.concat([tt, tt2], axis=1)
tt['TripType'] = tt_ind


#add total no of purchases
# tt_tot_prch = pd.DataFrame(pd.pivot_table(tat1, index = 'VisitNumber', values = 'ScanCount', aggfunc=np.sum))
# tt_tot_prch.columns = ['total_items']
tt_tot_prch =pd.pivot_table(tat1, index = 'VisitNumber', values = 'ScanCount', aggfunc=np.sum)


#add weeks
tat1['day']=1
tt_wkd = pd.pivot_table(tat1, index = 'VisitNumber', columns = 'Weekday', values = 'day', aggfunc=np.mean)
tt_ttype = pd.pivot_table(tat1, index = 'VisitNumber', values = 'TType', aggfunc=np.mean)
tt_wkd.columns = ['Weekday=' + i for i in tt_wkd.columns]
tt_wkd = tt_wkd.fillna(0).astype(np.int8)

tt_wkd['Weekday=weekend']=tt_wkd['Weekday=Saturday']+tt_wkd['Weekday=Sunday']

# tt = pd.concat([tt, tt_wkd, tt_ttype,tt_tot_prch], axis=1) # with weekday
# tt = pd.concat([tt, tt_ttype,tt_tot_prch], axis=1) #w/o w weekday
tt = pd.concat([tt, tt_ttype,tt_tot_prch, tt_wkd['Weekday=weekend']], axis=1) #with weekend only

#new vars noItmes
tt['SC=1'] = (tt['ScanCount']==1)*1
tt['SC=2'] = (tt['ScanCount']==2)*1
tt['SC=-1'] = (tt['ScanCount']==-1)*1
tt['SC<-1'] = (tt['ScanCount']<-1)*1
tt['SC>2'] = (tt['ScanCount']>2)*1
tt['SC=0'] = (tt['ScanCount']==0)*1

#new var noDepts
NoDeptsCount = list(np.zeros(tt2.shape[0]))
for i in tt2.columns:
    NoDeptsCount+= (tt2[i]!=0)*1
tt['NoDepts'] = NoDeptsCount

# unique, counts = np.unique(tt['NoDepts'], return_counts=True)
# np.asarray((unique, counts)).T


#new var #food
food_col = ['DepartmentDescription=COOK AND DINE',
'DepartmentDescription=SERVICE DELI',
'DepartmentDescription=BAKERY',
'DepartmentDescription=DSD GROCERY',
'DepartmentDescription=COMM BREAD',
'DepartmentDescription=FROZEN FOODS',
'DepartmentDescription=DAIRY',
'DepartmentDescription=SEAFOOD',
'DepartmentDescription=GROCERY DRY GOODS',
'DepartmentDescription=MEAT - FRESH & FROZEN',
'DepartmentDescription=PRE PACKED DELI',
'DepartmentDescription=PRODUCE',]

'DepartmentDescription=BAKERY',
# #( BREAD: ?not working - delete?)

tt['SC_food']=0
for i in food_col:
    tt['SC_food']+=tt[i]
tt['food_share'] = tt['SC_food']/tt['ScanCount']*(tt['ScanCount']>0)
tt['food_share'] = tt['food_share'].fillna(0)

tt['SC_food=1']=(tt['SC_food']==1)*1 #almost unnecessary

# tt['food_share>0.2'] = (tt['food_share']>=0.2)*1 #(?not working - delete?)


In [5]:
#adding wear

wear_col=[
'SHOES',
'BOYS WEAR',
'MENS WEAR',
'LADIESWEAR',
'SHEER HOSIERY',
'LADIES SOCKS',
'GIRLS WEAR, 4-6X  AND 7-14',
'BRAS & SHAPEWEAR',
'SLEEPWEAR/FOUNDATIONS',
'MENSWEAR',
'SWIMWEAR/OUTERWEAR']

tt['SC_wear']=0

for i in wear_col:
    tt['SC_wear']+=tt['DepartmentDescription='+i]
tt['wear_share'] = tt['SC_wear']/tt['ScanCount']*(tt['ScanCount']>0)
tt['wear_share'] = tt['wear_share'].fillna(0)

In [6]:
#adding other

beauty_col=[
'PERSONAL CARE',
'BEAUTY',
'HEALTH AND BEAUTY AIDS',]

remont_col = [
'PAINT AND ACCESSORIES',
'BEDDING',
'HOME DECOR',
'FURNITURE',
'LARGE HOUSEHOLD GOODS',]

house_col = [
'HOUSEHOLD CHEMICALS/SUPP',
'HOME MANAGEMENT',
'HOUSEHOLD PAPER GOODS',
'BATH AND SHOWER',]

glasses_col = [
'JEWELRY AND SUNGLASSES',
'OPTICAL - FRAMES',
'OPTICAL - LENSES',]

device_col = [
'HARDWARE',
'WIRELESS',
'ELECTRONICS',
'CAMERAS AND SUPPLIES',
'PLAYERS AND ELECTRONICS',]

pharma_col =[
'PHARMACY OTC',
'PHARMACY RX',]

pregnant_col=[
'INFANT CONSUMABLE HARDLINES',
'INFANT APPAREL',
'PLUS AND MATERNITY',]

categ = [beauty_col,remont_col,house_col,glasses_col,device_col,pharma_col,pharma_col,pregnant_col]
categ_names = ['beauty','remont','house','glasses','device','pharma','pharma','pregnant']

In [7]:
for j in range(len(categ)):
    tt['SC_'+categ_names[j]] = 0
    for i in categ[j]:
        tt['SC_'+categ_names[j]]+=tt['DepartmentDescription='+i]
    tt[categ_names[j]+'_share'] = tt['SC_'+categ_names[j]]/tt['ScanCount']*(tt['ScanCount']>0)
    tt[categ_names[j]+'_share'] = tt[categ_names[j]+'_share'].fillna(0)

In [8]:
# selectiong important UPC

UPC = pd.pivot_table(train_raw, index = 'Upc', values = 'ScanCount', columns = 'TripType', aggfunc=np.sum)
## здесь может быть также подсчет по аблолютным значениям и разделение на покупку и возврат

UPC = UPC.fillna(0)

UPC_cols = UPC.columns
UPC['count'] = 0
for col in UPC_cols: UPC['count']+=abs(UPC[col])
UPC['max'] = UPC[UPC_cols].apply(max, axis = 1)
UPC['max_share'] = UPC['max'] / UPC['count']
UPC = UPC.fillna(0)

# еще можно: частота покупок айтема, его доля в корзине, связь его покупок с чем-то другим, совместные покупки
# если два продукта купили вместе, какова вероятность быть такого-то типа визита

# UPC[(UPC['count']>100)&(UPC['count']<10000)]['max_share'].hist(bins=100)
# UPC[(UPC['count']>10)]['max_share'].hist(bins=100)
# plt.show()

# selecting neeged UPCs # a lot more to do here
UPC_cols = [int(i) for i in UPC[UPC['count']>200].index]
UPC_cols2 = ['UPC='+str(col) for col in UPC_cols]

In [9]:
# making UPC columns

tt_upc = train_raw
for col in UPC_cols:
    tt_upc['UPC='+str(col)]=np.zeros(tt_upc.shape[0], dtype = np.int8)
    tt_upc['UPC='+str(col)]=np.array((tt_upc['Upc']==col)*1*tt_upc['ScanCount'], dtype= np.int16)
    
tt_upc_test = test_raw
for col in UPC_cols:
    tt_upc_test['UPC='+str(col)]=np.zeros(tt_upc_test.shape[0], dtype = np.int8)
    tt_upc_test['UPC='+str(col)]=np.array((tt_upc_test['Upc']==col)*1*tt_upc_test['ScanCount'], dtype= np.int16)
    
tt_upc = tt_upc.groupby(['VisitNumber'])[UPC_cols2].sum()
tt_upc_test = tt_upc_test.groupby(['VisitNumber'])[UPC_cols2].sum()

In [134]:
# cartesian product for finding assotiative items

# import itertools
# a = list(itertools.product(UPC[UPC['count']>1000].index,UPC[UPC['count']>1000].index))
# a = pd.DataFrame(a)
# a['count']=0
# a['count'] = a.apply(lambda row: train_raw[train_raw['Upc']==row[0]].shape[0], axis = 1)

In [10]:
# # done 1 time

# tt4  = pd.pivot_table(tat1, values='ScanCount', index=['VisitNumber'],
#                    columns=['FinelineNumber'], aggfunc=np.sum)
# tt4 = tt4.fillna(0).astype(np.int8) 

# # tt4.to_csv(path_or_buf = tt4_save_name1, index = True)
# # runned 10 minutes


# # done each time (or 1 time)
tt4  = pd.pivot_table(tat1[tat1['TType']==1], values='ScanCount', index=['VisitNumber'],
                   columns=['FinelineNumber'], aggfunc=np.sum)
tt4 = tt4.fillna(0).astype(np.int8) 

# tt4.to_csv(path_or_buf = tt4_save_name1, index = True)


In [4]:
# # done each notebook reset
# probably not used anymore

# tt_4_copy = pd.read_csv(filepath_or_buffer=tt4_save_name1,dtype = dtypes_tt4)
# tt_4_copy = tt_4_copy.astype(np.int8)
# tt_4_copy.index = tt_4_copy.VisitNumber
# # tt_4_copy = tt_4_copy.drop(['VisitNumber'],1)
# tt_4_copy.columns = ['FinelineNumber=' + i for i in tt_4_copy.columns]
# tt_4_copy['VisitNumber'] = tt_4_copy.index
# # tt5 = pd.merge(tt_4_copy, tt, on=['VisitNumber])
# tt5 = pd.merge(tt_4_copy, tt, on=[tt['VisitNumber']])

In [11]:
# tt4_run = tt
tt4 = pd.concat([tt4, tt[tt['TType']==1]], axis=1)
tt4 = pd.concat([tt4,tt_upc], axis=1)

In [12]:
tt4_test = pd.pivot_table(tat1[tat1['TType']==0], values='ScanCount', index=['VisitNumber'],
                   columns=['FinelineNumber'], aggfunc=np.sum)
tt4_test = tt4_test.fillna(0).astype(np.int8) 
tt4_test = pd.concat([tt4_test, tt[tt['TType']==0]], axis=1)
tt4_test = pd.concat([tt4_test,tt_upc_test], axis=1)

In [13]:
# feature 24 hack

f24=[
'FURNITURE',                   
'BEDDING',                        
'HOME DECOR',                     
'HOME MANAGEMENT',                
'BATH AND SHOWER',             
'COOK AND DINE',]

f24 = ['DepartmentDescription='+i for i in f24]

for i in tt2.columns:
    tt4[i+'_share'] = tt4[i]/tt4['ScanCount']*(tt4['ScanCount']>0)*1
    tt4[i+'_share'] = tt4[i+'_share'].fillna(0)
    
tt4['f24']=0
for i in f24:
    tt4['f24']+= tt4[i]
tt4['f24_share'] = tt4['f24']/tt4['ScanCount']*(tt4['ScanCount']>0)*1
tt4['f24_share'] = tt4['f24_share'].fillna(0)


for i in tt2.columns:
    tt4_test[i+'_share'] = tt4_test[i]/tt4_test['ScanCount']*(tt4_test['ScanCount']>0)*1
    tt4_test[i+'_share'] = tt4_test[i+'_share'].fillna(0)

tt4_test['f24']=0
for i in f24:
    tt4_test['f24']+= tt4_test[i]
tt4_test['f24_share'] = tt4_test['f24']/tt4_test['ScanCount']*(tt4_test['ScanCount']>0)*1
tt4_test['f24_share'] = tt4_test['f24_share'].fillna(0)

###1.2 data split for CV

In [14]:
#data for modelling

train.data = tt4[tt4['TType']==1].drop(['VisitNumber','TripType','TType'],1) #'VisitNumber' ,'FinelineNumber=VisitNumber'
train.target = tt4[tt4['TType']==1]['TripType']

test.data = tt4_test.drop(['VisitNumber','TripType','TType'],1)


X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    train.data, train.target, test_size=0.2, random_state=42) # , random_state=np.random.RandomState()

# X_train, X_rest, y_train, y_rest = cross_validation.train_test_split(
#      train.data, train.target, test_size=0.4, random_state=42) # , random_state=np.random.RandomState()

# X_CV, X_test, y_CV, y_test = cross_validation.train_test_split(
#      X_rest, y_rest, test_size=0.5, random_state=42) # , random_state=np.random.RandomState()

In [15]:
# для вероятностного предсказания
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=True)
enc.fit(tt4[['TripType']])
b = enc.transform(tt4[['TripType']]).toarray()
c = np.unique(tt4[['TripType']])
y_proba = pd.DataFrame(data = b, columns = c)
# y_CV_proba = enc.transform(pd.DataFrame(y_CV.values)).toarray()
# y_CV_proba =  pd.DataFrame(data = y_CV_proba, columns = c)

# y_test_proba = enc.transform(pd.DataFrame(y_CV.values)).toarray()
y_test_proba =  pd.DataFrame(data = y_test, columns = c)

##2 Running models

###2.1 RF

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# rf = RandomForestClassifier(n_estimators=100, max_depth=None,
#      min_samples_leaf=3, random_state=42)
rf = RandomForestClassifier(n_estimators=100, max_depth=None,
     min_samples_leaf=3, random_state=42)
rf.fit(X_train, y_train)
curr_predict = rf.predict_proba(X_test)
# accuracy_score(curr_predict, y_CV.values, normalize=True)
log_loss(y_test, curr_predict)
# basic features 0.629
# + total items 0.6719
# + new #items vars 0.6729
# + noDepts 0.6726 (? not working)
# (weekday not working?)
# + food_total 0.6744
# + food share 0.6757 (could be optimized)
# + sc_food=1 0.6767
# + incl into food bakery 0.6748 (? not working)
# + incl into food service dely 0.6791
# + foodshare >=0.2 0.6774 (? not working)
# + incl into food cook and dine 0.6756 
# del var food_share>02 0.6769
# del weekday 0.6771
# + weekend 0.6792
# +Fineline 0.6599
# Fineline and run on selected features 0.6878 with imp>2*(10**(-4))
# Fineline and run on selected features 0.6828 with imp>2*(10**(-5))
# Fineline and run on selected features 0.6873 with imp>4*(10**(-4))
# n_estimators = 1000 0.6889

# 

#proba seed 42
# all vars 1.20509
# selecting vars with imp[imp[0]>10**(-5)] 1.0938

# all var, +wear 1.18509
# + all shares 1.10560
# + f24 hack 1.09885

1.0988556799461771

In [236]:
curr_predict = rf.predict_proba(X_test)

In [368]:
log_loss(y_test, curr_predict)

1.1510419659273867

In [677]:
# for i in y_train.unique():
#     if i not in y_CV.unique():
#         print(i) #14
y_CV[121721]=14
from sklearn.metrics import log_loss
y_true = y_CV
y_pred = predict_ll
log_loss(y_true, y_pred)

1.0267481570706887

In [ ]:
# analysis

# tt['TripType'].hist(bins=999)
# plt.show()

# s = 0
# for i in tt['TripType'].unique():
#     ds = tt[tt['TripType']==i].shape[0]/tt['TripType'].shape[0]
#     s = s + ds
#     print(ds, i)

###2.2 Xgboost and other

In [120]:
# Xgboost param
# check no of cores
param = {'max_depth':3, 'learning_rate':0.1, 'n_estimators':100, 'silent':True,
         'objective':'multi:softprob','nthread':1, 'gamma':0, 'min_child_weight':1,
         'max_delta_step':0, 'subsample':1, 'colsample_bytree':1, 'colsample_bylevel':1,
         'reg_alpha':0, 'reg_lambda':1, 'scale_pos_weight':1, 'base_score':1/34, 'seed':0, 'missing':None}

In [9]:
start_time = datetime.now()
xgb_model = xgb.XGBClassifier(**param).fit(np.array(X_train),np.array(y_train))
end_time = datetime.now()

In [ ]:
a = xgb_model.predict(np.array(X_test))
accuracy_score(a, y_test.values, normalize=True)

In [ ]:
duration = -(int(str(start_time)[14:16])*60+int(str(start_time)[17:19])) + (
    int(str(end_time)[14:16])*60+int(str(end_time)[17:19]))
xgb_model.booster().save_model('xgb_model_01')
xgb_model.booster().dump_model('xgb_model_01.dump', 'featmap')

In [26]:
# runtime analysis
# 50 \ 9
# 100 \ 19
# 200 \ 43
# 300 \ 71
# 1000 \ 247
# 5000 \ 1160

In [524]:
rf.predict(X_test[new_col])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [262]:
y_CV.shape[0]*0.0013

24.8755

###3 Result analysis

In [531]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(y_test.values, rf_new.predict(X_test[new_col])))
var_stats = pd.DataFrame(data = {'total':np.zeros(cm.shape[0])})

var_stats['total'] = cm.sum(axis = 1)
var_stats['predict'] = cm.sum(axis = 0)
var_stats['correct']=np.diag(cm)
var_stats['TP']=100*var_stats['correct']/var_stats['total']
var_stats['err']=var_stats['total']-var_stats['correct']

# row4: feature 7, err = 423
# row5:feature8

# to check feture name compare this with total
# unique, counts = np.unique(y_CV, return_counts=True)
# np.asarray((unique, counts)).T
var_stats.index= rf_new.classes_
var_stats

,total,predict,correct,TP,err
3,715,769,703,98.321678,12
4,63,11,11,17.460317,52
5,948,1025,798,84.177215,150
6,273,220,194,71.062271,79
7,1136,1051,778,68.485915,358
8,2456,2728,2220,90.390879,236
9,1906,2245,1583,83.053515,323
12,56,0,0,0.000000,56
14,1,0,0,0.000000,1
15,190,99,70,36.842105,120


##4 post-analysis run

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=100, max_depth=None,
#      min_samples_leaf=3, random_state=42)
rf = RandomForestClassifier(n_estimators=100, max_depth=None,
     min_samples_leaf=3, random_state=42)
rf.fit(X_train, y_train)
curr_predict = rf.predict_proba(X_CV)
# accuracy_score(curr_predict, y_CV.values, normalize=True)

In [61]:
curr_predict = rf.predict_proba(X_CV)

In [63]:
from sklearn.metrics import log_loss

In [72]:
data_target

NameError: name 'data_target' is not defined

In [98]:
rf.classes_

array([  3,   4,   5,   6,   7,   8,   9,  12,  14,  15,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 999], dtype=int64)

In [80]:
# log_loss(curr_predict, y_test)


In [148]:
# from sklearn.feature_extraction import DictVectorizer
# vec = DictVectorizer()

# a = tt4[['TripType']].to_dict(orient = 'records')
# b = vec.fit_transform(a).toarray()
# c = vec.get_feature_names()
# d = pd.DataFrame(data = b, columns = c)
# # tat = pd.concat([tat.drop(['Weekday','DepartmentDescription'],1),d],axis=1,join='inner')
# # new_col = c

In [ ]:
enc = enc.fit(np.unique(y_train))
y_CV_proba =  enc.transform(y_CV)

In [106]:
train.target.unique()

array([999,  30,  26,   8,  35,  41,  21,   6,  42,   7,   9,  39,  25,
        38,  15,  36,  20,  37,  32,  40,   5,   3,   4,  24,  33,  43,
        31,  27,  34,  18,  29,  44,  19,  23,  22,  28,  14,  12], dtype=int64)

In [89]:
rg

array([999,  30,  26, ...,  39,  39,   8], dtype=int64)

##5 Grid search

In [48]:
short_parameters = {'n_estimators':[10,20],
              'max_features':(1.0,'sqrt')}
# parameters = {'n_jobs':[-1],'criterion':['gini','entropy'],'n_estimators':[10,30,100,300,1000],
#               'max_features':(1.0,'sqrt','log2'),'max_depth':[None,3,6],'min_samples_leaf':[1,3,10,30],}
parameters = {'n_jobs':[1],'criterion':['gini'],'n_estimators':[30,100,300],
               'max_features':(1.0,'sqrt'),'min_samples_leaf':[1,3,7]}

In [49]:
from sklearn import grid_search
rf_grid = RandomForestClassifier()
clf = grid_search.GridSearchCV(estimator = rf_grid, param_grid = parameters)

In [50]:
clf.fit(X_train[new_col],y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'min_samples_leaf': [1, 3, 7], 'n_jobs': [1], 'n_estimators': [30, 100, 300], 'criterion': ['gini'], 'max_features': (1.0, 'sqrt')},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [51]:
scores = pd.DataFrame(clf.grid_scores_)

# pd.DataFrame(clf.grid_scores_)
scores['std'] = scores.apply(lambda row: np.std(row[2]), axis = 1)
scores['mean'] = scores.apply(lambda row: np.mean(row[2]), axis = 1)
scores = scores.drop([1,2],1)
scores = scores.sort(['mean', 'std'], ascending=[0, 1])
# посмотреть на счет устойчивости score по отношению к параметрам
scores

,0,std,mean
1,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 100, 'n_jobs': 1, 'max_features': 1.0}",0.000648,0.706658
11,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 300, 'n_jobs': 1, 'max_features': 'sqrt'}",0.003300,0.706364
2,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 300, 'n_jobs': 1, 'max_features': 1.0}",0.001097,0.706153
0,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 30, 'n_jobs': 1, 'max_features': 1.0}",0.001298,0.704202
10,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 100, 'n_jobs': 1, 'max_features': 'sqrt'}",0.003579,0.703942
5,"{'min_samples_leaf': 3, 'criterion': 'gini', 'n_estimators': 300, 'n_jobs': 1, 'max_features': 1.0}",0.002624,0.701729
4,"{'min_samples_leaf': 3, 'criterion': 'gini', 'n_estimators': 100, 'n_jobs': 1, 'max_features': 1.0}",0.002819,0.701119
3,"{'min_samples_leaf': 3, 'criterion': 'gini', 'n_estimators': 30, 'n_jobs': 1, 'max_features': 1.0}",0.002601,0.700963
9,"{'min_samples_leaf': 1, 'criterion': 'gini', 'n_estimators': 30, 'n_jobs': 1, 'max_features': 'sqrt'}",0.002771,0.696207
8,"{'min_samples_leaf': 7, 'criterion': 'gini', 'n_estimators': 300, 'n_jobs': 1, 'max_features': 1.0}",0.002780,0.690145


###6 Feature engineering

In [40]:
help(RandomForestClassifier())

Help on RandomForestClassifier in module sklearn.ensemble.forest object:

class RandomForestClassifier(ForestClassifier)
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and use averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |      Note: this parameter is tree-specific.
 |  
 |  max_features : int, float, string or None, optional (default="auto")
 |      The number of features to consider when looking for the best split:
 |  
 |      - If int, then consider `max_features` features at each split.
 |      - If 

In [45]:
for i in parameters: print(i)

max_depth
max_features
n_jobs
min_samples_leaf
n_estimators
criterion


In [544]:
pd.DataFrame(new_col).to_csv('new_col')

In [ ]:
feat8 vs feat7
DepartmentDescription=PERSONAL CARE	0.115073	0.007945
DepartmentDescription=IMPULSE MERCHANDISE	0.108536	0.024684

In [ ]:
C3.sort(['dif'], ascending=[0])

In [416]:
C3=pd.DataFrame()
C3['8'] = C2['mean']
C3['7'] = C1['mean']
C3['dif'] = (C3['8']+C3['7']>0.01)*1 *(C3['8'])/(C3['8']+C3['7'])


In [400]:
C2 = tt[tt['TripType']==8].describe().transpose().sort(['mean'], ascending=[0])/1.937258

In [ ]:
C1 = tt[tt['TripType']==7].describe().transpose().sort(['mean'], ascending=[0])/4.704798
C1

In [ ]:
# tt[tt['TripType']==8].describe().ix['mean','DepartmentDescription=COMM BREAD']
tt[tt['TripType']==8].describe().transpose().sort(['mean'], ascending=[0])/1.937258

In [ ]:
feat 7 and 8 diff 
1) comm bread 
2) FROZEN FOODS
3) slight: GROCERY DRY GOODS
4) MEAT - FRESH & FROZEN
5) PERSONAL CARE 1 vs 11
6) PRE PACKED DELI
7) PRODUCE
8) SERVICE DELI strong
9) ScanCount
10 ) IMPULSE MERCHANDISE 2 vs 10

In [344]:
tt[tt['TripType']==7]['food_share'].hist(bins=100)
plt.show()

In [421]:
train[train['ScanCount']==1]['TripType'].hist(bins=100)
plt.show()

In [286]:
train[train['TripType']==7]['ScanCount'].hist(bins=100)
plt.show()

In [429]:
tt[(tt['food_share>0.2']==1)&(tt['TripType']!=999)&(tt['TripType']!=-1)
  ]['TripType'].hist(bins=100)
plt.show()

In [320]:
LIQUOR,WINE,BEER
IMPULSE MERCHANDISE
COOK AND DINE

In [ ]:
tt[tt['TripType']==38][['ScanCount','SC_food','food_share']][1581:]

In [ ]:
train[train['VisitNumber']==107597]

###7 Feature importance

In [17]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]


# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]],))

# Plot the feature importances of the forest
# plt.figure()
# plt.title("Feature importances")
# plt.bar(range(X_train.shape[1]), importances[indices],
#        color="r", yerr=std[indices], align="center")
# plt.xticks(range(X_train.shape[1]), indices)
# plt.xlim([-1, X_train.shape[1]])
# plt.show()

Feature ranking:
1. feature 5265 (0.075335)
2. feature 5274 (0.033074)
3. feature 5273 (0.030355)
4. feature 5275 (0.025742)
5. feature 5271 (0.023658)
6. feature 5269 (0.020558)
7. feature 5278 (0.019099)
8. feature 5290 (0.019013)
9. feature 5213 (0.017123)
10. feature 5277 (0.016519)
11. feature 5535 (0.015850)
12. feature 5280 (0.015761)
13. feature 5532 (0.015710)
14. feature 5267 (0.015000)
15. feature 5575 (0.014141)
16. feature 5216 (0.013669)
17. feature 5289 (0.012575)
18. feature 5279 (0.012188)
19. feature 5220 (0.011978)
20. feature 5248 (0.011047)
21. feature 5272 (0.010664)
22. feature 5565 (0.010597)
23. feature 5567 (0.010553)
24. feature 5531 (0.010520)
25. feature 5539 (0.010261)
26. feature 5246 (0.010249)
27. feature 5572 (0.010207)
28. feature 5268 (0.009643)
29. feature 5284 (0.009310)
30. feature 5249 (0.008642)
31. feature 5283 (0.008428)
32. feature 5253 (0.008348)
33. feature 5585 (0.008183)
34. feature 5584 (0.008138)
35. feature 5256 (0.007993)
36. feature 

In [ ]:
# imp = pd.DataFrame(rf.feature_importances_,X_train[new_col].columns)
# new_col = imp[imp[0]>10**(-4)*4].sort([0], ascending=[0]).index
# imp[imp[0]>10**(-4)*4].sort([0], ascending=[0])

In [282]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# rf = RandomForestClassifier(n_estimators=100, max_depth=None,
#      min_samples_leaf=3, random_state=43)
# rf.fit(X_train, y_train)
# curr_predict = rf.predict_proba(X_test)
# log_loss(y_test, curr_predict)

1.201066170521889

In [18]:
imp = pd.DataFrame(rf.feature_importances_,X_train.columns)
# new_col = imp[imp[0]>10**(-4)*4].sort([0], ascending=[0]).index
# imp[imp[0]>10**(-4)*4].sort([0], ascending=[0])
imp = imp.sort([0], ascending=[0])
thresh_cost = 2.5*10**(-4)
new_col = imp[imp[0]>thresh_cost].index
print(imp[imp[0]>thresh_cost].shape)
imp[imp[0]>thresh_cost]

(234, 1)


,0
FinelineNumber,
ScanCount,0.075335
SC_food,0.033074
NoDepts,0.030355
food_share,0.025742
SC>2,0.023658
SC=-1,0.020558
wear_share,0.019099
pharma_share,0.019013
DepartmentDescription=DSD GROCERY,0.017123


In [655]:
from sklearn.ensemble import RandomForestClassifier
rf_new = RandomForestClassifier(n_estimators=150, max_depth=None,
     min_samples_leaf=2, random_state=43)
rf_new.fit(X_train[new_col], y_train)
curr_predict_new = rf_new.predict_proba(X_test[new_col])
log_loss(y_test, curr_predict_new)

0.90791686912379155

In [555]:
from sklearn.ensemble import RandomForestClassifier
rf_new2 = RandomForestClassifier(n_estimators=150, max_depth=None,
     min_samples_leaf=2, random_state=43)
rf_new2.fit(X_train[new_col2], y_train)
curr_predict_new2 = rf_new2.predict_proba(X_test[new_col2])
log_loss(y_test, curr_predict_new2)

0.92877433263111275

In [ ]:
#proba seed 42
# all vars 1.20509
# selecting vars with imp[imp[0]>10**(-5)] 1.0938

#proba seed 43
# all vars 1.20106
# selecting vars with imp[imp[0]>10**(-5)] 1.0953
# thresh 10**(-4) 1.0337
# thresh_cost = 3*10**(-4) 1.01230
# min_leaf=2 0.9819
# min_leaf=4 1.03214
# n_trees = 150 0.9789
# n_trees = 300 0.9732
# ntrees 150, + wear thresh_cost = 3*10**(-4) 0.97181
# + other beauty, pharma etc 0.94605
# thresh  2*10**(-4) 0.94737
# thresh 4*10**(-4) 0.94670
# + other shares 0.91735
# thresh 2*10**(-4) 0.9117
# deleted ['SC_food','food_share'] 0.91678 
# +f24, back food 0.91327 (?negative effect)
# thresh_cost = 2.5*10**(-4) 0.90791

In [ ]:
# feature - share of top1 category, share of top3 categories/number of categories with share > 10%

In [519]:
# col_to_del =['SC_food','food_share']
# new_col = list(new_col)
# for i in col_to_del:
#     new_col.remove(i)

In [20]:
y_train_no14 = y_train
y_train_no14[y_train_no14==14]=999
y_test_no14 = y_test
y_test_no14[y_test_no14==14]=999

from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV

rf_clbr = RandomForestClassifier(n_estimators=150, min_samples_leaf=2, n_jobs=-1)
calibrated_clf = CalibratedClassifierCV(rf_clbr, method='isotonic', cv=5)
calibrated_clf.fit(X_train[new_col], y_train_no14)
ypreds = calibrated_clf.predict_proba(X_test[new_col])
print("loss WITH calibration : ", log_loss(y_test, ypreds))

loss WITH calibration :  0.845235041935


In [686]:

# sigmoid loss WITH calibration :  0.84982

print("loss WITH calibration : ", log_loss(y_test_no14, ypreds))

loss WITH calibration :  0.849820154921


In [684]:
y_test_no14 = y_test
y_test_no14[y_test_no14==14]=999
y_test_no14.unique()

array([  8,  36,  39, 999,   9,  37,  40,   7,   5,  25,  30,   3,  42,
        24,  44,  43,   6,   4,  34,  41,  35,  32,  31,  33,  22,  29,
        38,  21,  15,  18,  28,  27,  19,  12,  23,  26,  20,  99], dtype=int64)

##8 Submit

In [19]:
test.data = tt4_test[new_col]

In [22]:
# check for differ-ces in train/test and predict
a = tt4_test[new_col].columns
b = tt4[new_col].columns
for i in a:
    if i not in b: print(i)
for i in b:
    if i not in a: print(i)

In [21]:
from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV

# predict calibrated
train.target_no14 = train.target
train.target_no14[train.target_no14==14]=999

rf_clbr_pr = RandomForestClassifier(n_estimators=150, min_samples_leaf=2, n_jobs=-1)
calibrated_clf_pr = CalibratedClassifierCV(rf_clbr_pr, method='isotonic', cv=5)
calibrated_clf_pr.fit(train.data[new_col], train.target)

# print("loss WITH calibration : ", log_loss(y_test, ypreds))

CalibratedClassifierCV(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
            cv=5, method='isotonic')

In [22]:
ypreds_pr = calibrated_clf_pr.predict_proba(test.data[new_col])
predict_final = ypreds_pr
predict_final = pd.DataFrame(data = predict_final, columns = calibrated_clf_pr.classes_)

In [507]:
#predict
from sklearn.ensemble import RandomForestClassifier
rf_predict = RandomForestClassifier(n_estimators=150, max_depth=None,
     min_samples_leaf=2, random_state=43)
rf_predict.fit(train.data[new_col], train.target)


predict_final =  rf_predict.predict_proba(test.data[new_col])
predict_final = pd.DataFrame(data = predict_final, columns = rf_predict.classes_)
# predict_ll = rf.predict_proba(X_CV[new_col])

In [32]:
submission = sample_submission.copy()
for i in submission.columns:
    if i[:9] == 'TripType_':
        if int(i[9:])==14: pass
        else: submission[i] = predict_final[int(i[9:])]
submission.to_csv('submission', index = False)

In [31]:
submission.columns

Index(['VisitNumber', 'TripType_3', 'TripType_4', 'TripType_5', 'TripType_6',
       'TripType_7', 'TripType_8', 'TripType_9', 'TripType_12', 'TripType_14',
       'TripType_15', 'TripType_18', 'TripType_19', 'TripType_20',
       'TripType_21', 'TripType_22', 'TripType_23', 'TripType_24',
       'TripType_25', 'TripType_26', 'TripType_27', 'TripType_28',
       'TripType_29', 'TripType_30', 'TripType_31', 'TripType_32',
       'TripType_33', 'TripType_34', 'TripType_35', 'TripType_36',
       'TripType_37', 'TripType_38', 'TripType_39', 'TripType_40',
       'TripType_41', 'TripType_42', 'TripType_43', 'TripType_44',
       'TripType_999'],
      dtype='object')

array([  3,   4,   5,   6,   7,   8,   9,  12,  15,  18,  19,  20,  21,
        22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,
        35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 999], dtype=int64)

In [271]:
sum((test.data.index == sample_submission['VisitNumber']))

95674

In [272]:
# submission = sample_submission.copy()
# submission.index = test.data.index
# for i in submission.columns:
#     if i[:9] == 'TripType_':
#         submission[i] = (predict_final==int(i[9:]))*1
# submission.index = sample_submission.index
# submission.to_csv('submission', index = False)

In [ ]:
tt.columns

In [401]:
tt4.shape

(95674, 5518)

In [ ]:
#по каждому классу визита - средняя доля каждого отдела в чеке, затем посмотреть, какие сочетания классов встречаются
#для каждого типавизита для feature engineering

#что на счет мультиколлинеарности? 

# замена кол-ва покупок по категории их долей в общем кол-ве покупок

In [606]:
RA = tt2.copy()
# RA['VisitNumber']=tt['VisitNumber']
RA['TripType']=tt['TripType']  

RA_av = tt2.copy()

RA_av['ScanCount'] = 0
for i in RA_av.columns:
    RA_av['ScanCount']+=RA_av[i]
    
RA_av['VisitNumber'] = tt['VisitNumber']
RA_av['TripType'] = tt['TripType']
RA_av = RA_av[RA_av['TripType']!=-1]
RA_av = RA_av.groupby(['VisitNumber']).sum()

In [607]:
RA_gb = RA[RA['TripType']!=-1].groupby(['TripType']).sum()
RA_gb.columns = [i[22:] for i in RA_gb.columns]
for i in RA_gb.columns:
    RA_gb[i]=RA_gb[i]/RA_gb[i].sum()*1000
RA_gb = RA_gb.transpose()

In [608]:
for col in tt2.columns:
    RA_av[col] = RA_av[col]/RA_av['ScanCount']*(RA_av['ScanCount']>0)*1
    RA_av[col]=RA_av[col].fillna(0)

In [609]:
RA_av = RA_av.groupby(['TripType']).mean()
RA_av_SC = RA_av['ScanCount']
RA_av = RA_av.drop(['ScanCount'],1)*1000
RA_av.columns = [i[22:] for i in RA_av.columns]
RA_av = RA_av.transpose()

In [610]:
RA_av

TripType,3,4,5,6,7,8,9,12,14,15,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,999
1-HR PHOTO,0.137250,0.000000,0.036287,0.217524,0.021732,0.000000,3.187518,0.000000,0.000000,2.790559,0.000000,0.666667,0.000000,0.392068,57.412254,0.000000,0.172480,0.265910,0.000000,0.000000,0.000000,0.000000,0.000000,0.420875,0.000000,0.126743,0.000000,0.061576,0.211677,0.000000,0.053657,0.000000,0.004078,0.171527,0.397253,0.114679,0.000000,1.954050
ACCESSORIES,0.091500,0.289017,0.204951,0.130514,0.105203,0.092509,5.498908,0.000000,0.000000,0.864860,0.639053,0.933333,0.771847,0.377353,0.089799,0.000000,0.540863,2.731964,0.331345,0.104382,1.059103,0.164962,14.270346,0.631313,0.501413,0.198756,0.193170,0.097154,0.628531,0.265819,0.179831,0.520134,0.301833,6.574958,3.776383,1.703978,1.912835,0.181870
AUTOMOTIVE,1.212371,0.602119,1.286967,0.717828,0.645945,0.890826,30.778436,0.907240,0.000000,0.394391,3.388556,0.981481,310.231556,0.626811,2.736434,0.000000,2.523671,1.468879,8.398860,2.196943,3.330206,6.196475,2.416302,1.657447,1.675087,2.245751,4.570534,1.327060,2.113789,0.916222,0.941170,2.363685,1.157219,12.166702,11.692908,10.601796,5.664557,0.679688
BAKERY,0.256199,4.753877,1.467126,2.684279,26.872932,11.277170,1.289976,4.724153,0.000000,16.891968,3.036917,0.478355,2.183876,0.972281,1.935638,0.000000,1.798009,1.932979,2.507070,2.674599,2.059509,1.649681,1.679647,1.402918,2.696975,2.745049,3.717166,8.292623,2.141391,8.828380,10.698527,7.867043,5.208015,2.764962,2.193456,4.031861,2.624490,2.946466
BATH AND SHOWER,0.411749,0.000000,0.584645,0.580788,0.100003,0.418003,6.018421,3.553726,0.000000,0.674920,1.516772,2.431746,1.557711,1.144046,2.655254,0.000000,32.034611,2.819804,3.253136,0.807196,4.697865,1.351772,2.138522,0.210438,0.819959,3.535503,1.865792,0.631708,2.464428,0.671855,0.366079,1.625255,1.583457,13.887632,10.314259,8.494811,8.657878,0.459925
BEAUTY,1.083618,3.239403,4.849070,1.929054,1.058407,22.757493,3.044432,4.782042,0.000000,3.929587,6.037758,3.442328,3.883245,3.434450,5.383434,3.237410,9.085761,8.444032,4.859511,4.061235,8.344439,10.247210,5.174763,2.278770,5.361277,5.282739,7.387100,1.286494,86.115084,2.272295,2.687257,11.167639,7.349529,10.534900,10.563269,18.909829,20.890387,2.889583
BEDDING,0.251624,0.000000,0.161478,0.228400,0.076719,0.061673,3.842045,0.168976,0.000000,0.164735,0.722092,1.047619,0.523286,0.496383,1.547534,1.798561,38.748970,0.949625,0.116945,0.844983,7.048534,0.293266,0.805231,0.120250,0.459670,0.417502,0.237387,0.110837,0.589556,0.244518,0.120085,0.841900,0.568147,3.770922,3.814422,1.600091,1.876014,0.259839
BOOKS AND MAGAZINES,0.217312,0.000000,0.355255,0.108427,0.212774,0.246690,3.020217,0.232342,0.000000,0.505070,2.561820,0.755556,1.046572,14.821884,0.685763,0.000000,0.575795,0.813527,0.521869,0.728752,0.451671,0.384911,0.393154,0.000000,0.224424,0.362459,0.040906,0.363693,0.485401,0.214690,0.275650,0.514959,0.377821,0.794974,3.350865,2.365078,1.563159,0.064148
BOYS WEAR,0.480373,0.000000,0.465777,0.097886,0.070369,0.142190,8.230437,0.836431,0.000000,0.457163,2.696047,0.148148,0.549451,1.232245,1.875124,0.513875,1.017042,24.831629,0.826857,0.304942,1.076752,1.472069,4.793424,0.000000,1.995188,0.565266,0.285615,0.516927,0.601203,0.186990,0.307972,0.884209,0.735859,8.034189,8.921058,3.357940,5.393057,0.224518
BRAS & SHAPEWEAR,0.000000,0.000000,0.152017,0.000000,0.028251,0.061673,4.233587,0.148699,0.000000,0.543200,0.000000,2.444444,0.261643,0.234009,0.412968,0.000000,0.898695,14.717658,0.090367,0.097991,0.573275,0.057737,1.536184,0.210438,0.682507,0.253485,0.424973,0.084448,0.442873,0.199758,0.140578,0.543695,0.429874,2.328951,3.083694,2.061064,3.700981,0.073643


In [611]:
RA_av_SC

TripType
3       3.998902
4       6.005780
5       6.775528
6       6.192639
7       9.409597
8       3.874517
9       3.597633
12     18.572491
14     41.500000
15     18.408998
18     12.469945
19      6.821333
20     11.161695
21     15.485179
22      8.431034
23      4.589928
24     16.440015
25     15.823148
26     11.757455
27     16.529936
28     12.081301
29     10.692841
30      9.648474
31      6.252525
32     16.602823
33     17.443346
34     16.673157
35     15.636453
36     16.314143
37     31.138451
38     23.883929
39     21.500404
40     66.984013
41     20.751286
42     23.387513
43     16.059633
44     39.243471
999    -0.420654
Name: ScanCount, dtype: float64

In [651]:
col = 39
np.round(RA_gb.sort([col],ascending=[0])[col])
np.round(RA_av.sort([col],ascending=[0])[col])

GROCERY DRY GOODS              73
DSD GROCERY                    69
PERSONAL CARE                  41
DAIRY                          33
PRODUCE                        29
PHARMACY OTC                   26
HOUSEHOLD CHEMICALS/SUPP       25
IMPULSE MERCHANDISE            23
FROZEN FOODS                   22
HOUSEHOLD PAPER GOODS          18
CANDY, TOBACCO, COOKIES        15
COMM BREAD                     14
BEAUTY                         11
PETS AND SUPPLIES               9
SERVICE DELI                    9
BAKERY                          8
PRE PACKED DELI                 7
INFANT CONSUMABLE HARDLINES     6
LIQUOR,WINE,BEER                5
MEAT - FRESH & FROZEN           4
MENS WEAR                       4
COOK AND DINE                   4
HOME MANAGEMENT                 3
CELEBRATION                     3
OFFICE SUPPLIES                 3
SHOES                           2
AUTOMOTIVE                      2
LAWN AND GARDEN                 2
HARDWARE                        2
FINANCIAL SERV

In [582]:
RA_av = RA_av.groupby(['VisitNumber']).sum()

1244

In [ ]:
for i in train.data.columns: print(i)

In [554]:
new_col2 = list(new_col)
for i in tt2.columns:
    if i in new_col2:
        new_col2.remove(i)

In [ ]:
# какой подход лучше - по сумме покупок в чеке / по доле от кол-ва в чеке / и так и так / с учетом среднего количества покупок
# данного товара

# nn I hit 0.66 with a single nn, just train for a very long time (~200 epochs) with some learn rate decay
# That's without any regularization either. That was using some custom neural net code on CPU.

#I use lasagne and train the nn on GPU, my features are very different from yours,
#all of my features are non-label encoded variables.

# изучить предыдущие чемпионаты для FE

# проревьюить весь путь с начала




In [648]:
# различить 39 и 7 классы error 132 on test set


cm.index = rf.classes_
cm.columns = rf.classes_
cm

,3,4,5,6,7,8,9,12,14,15,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,999
3,703,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5
4,0,11,49,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
5,0,0,798,2,0,31,9,0,0,0,0,0,1,1,1,0,2,9,0,0,0,0,2,0,2,1,3,2,13,4,2,60,5,0,0,0,0,0
6,1,0,0,194,9,26,3,0,0,1,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,4,1,2,2,26,1,0,0,0,0,0
7,0,0,2,1,778,114,7,0,0,0,0,0,0,0,0,0,2,2,0,1,1,0,1,2,1,2,2,10,3,55,17,132,3,0,0,0,0,0
8,0,0,58,6,54,2220,58,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,20,2,8,2,9,3,6,5,0,0,0,0,0,1
9,17,0,12,1,14,151,1583,0,0,2,0,4,0,5,14,4,15,37,0,0,0,0,20,15,7,1,0,1,0,0,1,2,0,0,0,0,0,0
12,0,0,1,0,5,0,6,0,0,0,0,0,0,0,0,0,3,1,0,7,0,0,0,0,0,5,0,2,2,3,2,17,2,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
15,0,0,3,0,7,12,6,0,0,70,1,0,0,10,1,0,6,0,0,2,1,0,1,0,4,5,0,8,3,5,4,25,11,0,5,0,0,0


In [660]:
from sklearn.preprocessing import LabelEncoder
LabelEncoder().fit_transform(y_test.values)

array([ 5, 28, 31, ...,  5, 37,  3], dtype=int32)

In [ ]:
# could be interesting
#https://www.kaggle.com/sushanttripathy/otto-group-product-classification-challenge/wrapper-for-models-ensemble/code